In [1]:
import pandas as pd
import numpy as np

In [29]:
class feature_engine:
    
    pd = __import__('pandas')
    datetime = __import__('datetime')
    mcal = __import__('pandas_market_calendars')
    np = __import__('numpy')
    
    def __init__(self):
        print('feature_engine V.0.1 \nImported pandas,datetime,pandas_market_calendars,numpy packages')
        
    def calc_price_change(self,tweet_time,financial_data,time_col_name,price_col_name,interval,amount,method='both',
                          logdiff=True):
        #remove seconds to match financial data format
        tweet_time=tweet_time.replace(second=0)
        #default baseline is set for five minutes before tweet
        tweet_time_5min_before=tweet_time - self.datetime.timedelta(0,0,0,0,5)
        
        tweet_price=financial_data[financial_data[time_col_name]==tweet_time_5min_before][price_col_name]
        if len(tweet_price)==0:
            return float('NaN')
        tweet_price.reset_index(inplace=True,drop=True)

        def calculation(financial_data,time_col_name,new_time,price_col_name,tweet_price,direction,logdiff):
            
            other_price=financial_data[financial_data[time_col_name]==new_time][price_col_name]
            other_price.reset_index(inplace=True,drop=True)
            if len(other_price)==0:
                return float('NaN')
            elif logdiff:
                if direction=='forward':
                    return (self.np.log(other_price.iloc[0]) - self.np.log(tweet_price.iloc[0]))
                elif direction=='backward':
                    return (self.np.log(tweet_price.iloc[0]) - self.np.log(other_price.iloc[0]))
            else:
                if direction=='forward':
                    return ((other_price.iloc[0]/tweet_price.iloc[0])-1)
                elif direction=='backward':
                    return ((tweet_price.iloc[0]/other_price.iloc[0])-1)

        if (method=='both') or (method=='forward'):  
            if interval=='minutes':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,amount)
            elif interval=='hours':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,0,amount)
            elif interval=='days':
                time_forward=tweet_time + self.datetime.timedelta(amount,0,0,0,0)
            pct_change_forward=calculation(financial_data,time_col_name,time_forward,price_col_name,tweet_price,'forward',
                                          logdiff)

        if (method=='both') or (method=='backward'): 
            if interval=='minutes':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,amount)   
            elif interval=='hours':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,0,amount) 
            elif interval=='days':
                time_backward=tweet_time - self.datetime.timedelta(amount,0,0,0,0)
            pct_change_backward=calculation(financial_data,time_col_name,time_backward,price_col_name,tweet_price,'backward'
                                           ,logdiff)
        
        if method == 'both':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_pct_change':pct_change_forward,
                          f'{amount}_{interval}_backward_pct_change':pct_change_backward},index=[0])
        elif method=='backward':
            return self.pd.DataFrame({f'{amount}_{interval}_backward_pct_change':pct_change_backward},index=[0])
        elif method=='forward':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_pct_change':pct_change_forward},index=[0])
        
    def create_pricechg_columns(self,twitter_data,tweet_time_col_name,financial_data,fin_time_col_name,price_col_name,
                       interval_amount_dict,method='both',logdiff=True):
        from tqdm import tqdm
        new_columns=self.pd.DataFrame()
        for i, row in tqdm(twitter_data.iterrows()):
            temp_df=self.pd.DataFrame()
            for interval, amount in interval_amount_dict:
                new_df=self.calc_price_change(twitter_data[tweet_time_col_name].loc[i],financial_data,fin_time_col_name,
                                                   price_col_name,interval,amount,method,logdiff)
                if not isinstance(new_df,float):
                    temp_df=self.pd.concat([temp_df,new_df],axis=1)
            for item in temp_df.columns:
                new_columns.at[i,item] = temp_df.loc[0,item]
        return new_columns
    
    def mean_encoding_tocolumn(self,features_df,categorical_column,target_column):
        features_df=features_df[[categorical_column,target_column]]
        grouped=features_df.groupby([categorical_column]).mean()
        features_df=features_df.merge(grouped,on=categorical_column)
        return features_df[f'{target_column}_y']
    
    def mean_encoding_todict(self,features_df,categorical_column,target_column):
        features_df=features_df[[categorical_column,target_column]]
        return features_df.groupby([categorical_column]).mean()
    
    def calculate_sum_volume(self,tweet_time,financial_data,time_col_name,volume_col_name,interval,amount):
        #remove seconds to match financial data format
        tweet_time=tweet_time.replace(second=0)
#         default baseline is set for five minutes before tweet
        tweet_time_5min_before=tweet_time - self.datetime.timedelta(0,0,0,0,5)
        
        def calculation(financial_data,time_col_name,time_forward,volume_col_name,tweet_time):
            forward_volume_sum=sum(financial_data.loc[(financial_data[time_col_name]<=time_forward) & 
                                        (financial_data[time_col_name]>=tweet_time)][volume_col_name])
            return forward_volume_sum

        if interval=='minutes':
            time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,amount)
#             time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,amount)
        elif interval=='hours':
            time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,0,amount)
#             time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,0,amount)
        elif interval=='days':
            time_forward=tweet_time + self.datetime.timedelta(amount,0,0,0,0)
#             time_backward=tweet_time - self.datetime.timedelta(amount,0,0,0,0)
        vol_sum=calculation(financial_data,time_col_name,time_forward,volume_col_name,tweet_time_5min_before)

        return self.pd.DataFrame({f'{amount}_{interval}_forward_vol_sum':vol_sum},index=[0])
        
    def create_volumesum_columns(self,twitter_data,tweet_time_col_name,financial_data,fin_time_col_name,volume_col_name,
                       interval_amount_dict):
        from tqdm import tqdm
        new_columns=self.pd.DataFrame()
        for i, row in tqdm(twitter_data.iterrows()):
            temp_df=self.pd.DataFrame()
            for interval, amount in interval_amount_dict:
                new_df=self.calculate_sum_volume(twitter_data[tweet_time_col_name].loc[i],financial_data,fin_time_col_name,
                                                   volume_col_name,interval,amount)
                if not isinstance(new_df,float):
                    temp_df=self.pd.concat([temp_df,new_df],axis=1)
            for item in temp_df.columns:
                new_columns.at[i,item] = temp_df.loc[0,item]
        return new_columns
    
    def fill_missing_fin_data(self,ticker_col_name,stock_exchg_name,start_date,end_date,frequency,fin_time_colname,
                          financial_data,volume_colname,price_colname):
        ticker_name=financial_data.iloc[0][ticker_col_name]
        stock_exchg = self.mcal.get_calendar(stock_exchg_name, open_time=self.datetime.time(5, 30), 
                                             close_time=self.datetime.time(12, 0))
        daterange = stock_exchg.schedule(start_date, end_date)
        dates=self.pd.DataFrame(self.mcal.date_range(daterange, frequency).tz_convert(None),columns=[fin_time_colname])

#         financial_data[fin_time_colname] = self.pd.to_datetime(financial_data[fin_time_colname], utc = True)

        financial_data=dates.merge(financial_data,on=fin_time_colname,how='left')
        financial_data[ticker_col_name]=ticker_name

        financial_data['SYM_SUFFIX'].fillna(0,inplace=True)

        financial_data[volume_colname].fillna(0,inplace=True)
        financial_data[price_colname].interpolate(inplace=True)
        return financial_data
    
    def token_matrix(self,text_column,financial_topic_words):
        
        def check_for_word(token_vector,word):
            if word in token_vector:
                return 1
            else:
                return 0
            
        new_df=self.pd.DataFrame()
        for topic_word in financial_topic_words:
            new_df[topic_word]=text_column.apply(lambda x:check_for_word(x.lower().split(),topic_word))
            
        return new_df
    
    def diff_from_meanlog(self,df,date_colname,numeric_colname):
        #This function returns the difference of logs between the original value and that month's average
        ins_df=df[[date_colname,numeric_colname]].copy().set_index(date_colname,drop=True).astype(int).resample("M").mean()
        ins_df[numeric_colname]=self.np.log(ins_df[numeric_colname])
        ins_df['month']=ins_df.index.map(str).str[:7]
        df['month']=df[date_colname].map(str).str[:7]
        averages=df[[date_colname,'month']].merge(ins_df,on='month',how='left')[numeric_colname]
        diff=self.np.log(df[numeric_colname])-averages
        return diff

In [3]:
financial_data=pd.read_csv('04052009-18122019-SPYDIA-minutedata.csv',parse_dates=['DATETIME'])
twitter_data=pd.read_csv('trump_tweets_reducted_after_nlp.csv',index_col=0,parse_dates=['created_at_utc'])

C:\Users\natan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5,6,13,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
sp500_data=financial_data[financial_data['SYM_ROOT']=='SPY']
dow_data=financial_data[financial_data['SYM_ROOT']=='DIA']

In [30]:
feat_eng = feature_engine()

feature_engine V.0.1 
Imported pandas,datetime,pandas_market_calendars,numpy packages


In [6]:
twitter_data['retweet_count']=feat_eng.diff_from_meanlog(twitter_data,'created_at_utc','retweet_count')
twitter_data['favorite_count']=feat_eng.diff_from_meanlog(twitter_data,'created_at_utc','favorite_count')

C:\Users\natan\Anaconda3\lib\site-packages\pandas\core\series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
sp500_data=feat_eng.fill_missing_fin_data('SYM_ROOT','NYSE','2009-05-04','2019-12-18','1min','DATETIME',
                          sp500_data,'SIZE','PRICE')

In [8]:
dow_data=feat_eng.fill_missing_fin_data('SYM_ROOT','NYSE','2009-05-04','2019-12-18','1min','DATETIME',
                          dow_data,'SIZE','PRICE')

In [50]:
tweets_finance_volume=feat_eng.create_volumesum_columns(twitter_data[['created_at_utc']],'created_at_utc',
                                                        dow_data[['DATETIME','SIZE']],'DATETIME','SIZE',
                       [('minutes',1),('minutes',5),('minutes',10),('minutes',15),('minutes',30),('hours',1),
                        ('hours',3)])

36732it [3:19:26,  3.46it/s] 


sum of volume will be log difference from mean

In [51]:
tweets_finance_volume=tweets_finance_volume.join(twitter_data[['created_at_utc']])

In [52]:
for column in tweets_finance_volume.columns[:-2]:
    tweets_finance_volume[column]=feat_eng.diff_from_meanlog(tweets_finance_volume,'created_at_utc',column)

In [53]:
tweets_finance_volume=tweets_finance_volume.replace([np.inf, -np.inf], float('nan'))
tweets_finance_volume.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36732 entries, 0 to 36731
Data columns (total 9 columns):
1_minutes_forward_vol_sum     9646 non-null float64
5_minutes_forward_vol_sum     9720 non-null float64
10_minutes_forward_vol_sum    9789 non-null float64
15_minutes_forward_vol_sum    9832 non-null float64
30_minutes_forward_vol_sum    9909 non-null float64
1_hours_forward_vol_sum       10044 non-null float64
3_hours_forward_vol_sum       36732 non-null float64
created_at_utc                36732 non-null datetime64[ns]
month                         36732 non-null object
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 4.1+ MB


In [54]:
tweets_finance=feat_eng.create_pricechg_columns(twitter_data[['created_at_utc']],'created_at_utc',dow_data,
                                                'DATETIME','PRICE',[('minutes',1),('minutes',5),('minutes',10),
                                                                    ('minutes',15),('minutes',30),('hours',1),
                                                                    ('hours',3),('hours',6)],
                                                                    method='forward')

36732it [11:05:01,  7.88it/s]  


In [55]:
twitter_data_with_finance=twitter_data.join(tweets_finance)

In [56]:
twitter_data_with_finance=twitter_data_with_finance.join(tweets_finance_volume.drop(['created_at_utc','month'],1))

In [57]:
twitter_data_with_finance.drop(['coordinates','extended_entities','geo','in_reply_to_screen_name',
                                             'in_reply_to_status_id','in_reply_to_user_id','is_quote_status','is_retweet',
                                             'lang','place','possibly_sensitive','quoted_status','quoted_status_id','id',
                                             'retweeted_status','retrieved_utc'],1,inplace=True)

In [58]:
twitter_data_with_finance=twitter_data_with_finance.join(pd.get_dummies(twitter_data_with_finance['source']))

In [59]:
twitter_data=pd.DataFrame()
financial_data=pd.DataFrame()
dow_data=pd.DataFrame()

In [60]:
twitter_data_with_finance=twitter_data_with_finance.drop(['source','truncated'],1)

In [61]:
topic_words=['china','chinese','nafta','trade','trades','trading','tariff','tariffs','opec','usmca','xi','jinping','sanctions',
            'market','markets','stock','stocks','financial','investment','dow','nasdaq','500','wall street','wall st',
            'unemployment','jobs','labor',
            'manufacture','manufacturers','consumer','consumers','bank','banks','factories','business','businesses'
               ,'corporate','corporates','industry','industries','product','agriculture','agricultural','products',
            'rate','rates','reserve','inflation','currency','depreciating','depreceate','fed','federal reserve',
            'deal','deals',
             'dollar','dollars','$',
             'billion','billions','gdp','growth',
             'revenue','economy','economies','economist','economic','economists','money',
             'companies',
             'price','prices',
             'cents','cent','purchase',
             'depletion','regulation',
             '401(k)','trillions','recession','depression',
            'taxes','taxation','tax','debt','deficit','spending','refinance','finance','savings','deficits','bankruptcy',
              'spend','cost','costs','subsidizing','subsidize',
            'iran','nuclear','wall','military','daca','bill','danger','conflict','rockets','russia',
                 'middle east','ukrainian','ukraine','isis','syria','border','russian','investigation','kim','jong','un',
                'caravan','sanctions','corona','virus']

In [62]:
financial_words_matrix=feat_eng.token_matrix(twitter_data_with_finance['text'],topic_words)

In [63]:
twitter_data_with_finance=twitter_data_with_finance.join(financial_words_matrix)

In [ ]:
# twitter_data_with_finance['trump_president']=twitter_data_with_finance['created_at_utc']>='2016-11-08 22:00:00'

In [ ]:
# twitter_data_with_finance.drop('text',1,inplace=True)

In [ ]:
# twitter_data_with_finance=twitter_data_with_finance.join(pd.get_dummies(twitter_data_with_finance['lda_topic']))

In [ ]:
# twitter_data_with_finance_for_analysis.drop('lda_topic',1,inplace=True)

In [64]:
twitter_data_with_finance.to_csv('twitter_data_with_dow_for_analysis.csv')

In [ ]:
# for x in twitter_data_with_finance_for_analysis[twitter_data_with_finance_for_analysis['currency']==1]['text']:
#     print(x)